In [ ]:
{
    import type {ReactElement, Source} from 'shared/ReactElementType';
    import type {
      ReactFragment,
      ReactPortal,
      RefObject,
      ReactEventResponder,
      ReactEventResponderInstance,
      ReactFundamentalComponent,
      ReactScope,
    } from 'shared/ReactTypes';
    import type {RootTag} from 'shared/ReactRootTags';
    import type {WorkTag} from 'shared/ReactWorkTags';
    import type {TypeOfMode} from './ReactTypeOfMode';
    import type {SideEffectTag} from 'shared/ReactSideEffectTags';
    import type {ExpirationTime} from './ReactFiberExpirationTime';
    import type {UpdateQueue} from './ReactUpdateQueue';
    import type {ContextDependency} from './ReactFiberNewContext';
    import type {HookType} from './ReactFiberHooks';
    import type {SuspenseInstance} from './ReactFiberHostConfig';
    
}

import 引入代码作用

**ReactElementType.js** 

```Source --> [fileName,lineNumber]```
```ReactElement --> [$$typeof,type,key,ref,props,_owner, //DEV _store:{validated},_self,_shadownChildren,_source]```

`Source` 是用于出错时判断文件，行数。
`ReactElement` 是`React` 生成 `Element` 的通用数据。

**ReactTypes.js**

定义react相关元素的结构。

**HookType**
```
export type HookType =
  | 'useState'
  | 'useReducer'
  | 'useContext'
  | 'useRef'
  | 'useEffect'
  | 'useLayoutEffect'
  | 'useCallback'
  | 'useMemo'
  | 'useImperativeHandle'
  | 'useDebugValue'
  | 'useResponder';
```
... 同理后续文件


In [ ]:
{
    import invariant from 'shared/invariant';
    import warningWithoutStack from 'shared/warningWithoutStack';
    import {
      enableProfilerTimer,
      enableFundamentalAPI,
      enableUserTimingAPI,
      enableScopeAPI,
    } from 'shared/ReactFeatureFlags';
    import {NoEffect, Placement} from 'shared/ReactSideEffectTags';
    import {ConcurrentRoot, BatchedRoot} from 'shared/ReactRootTags';
    import {
      IndeterminateComponent,
      ClassComponent,
      HostRoot,
      HostComponent,
      HostText,
      HostPortal,
      ForwardRef,
      Fragment,
      Mode,
      ContextProvider,
      ContextConsumer,
      Profiler,
      SuspenseComponent,
      SuspenseListComponent,
      DehydratedFragment,
      FunctionComponent,
      MemoComponent,
      SimpleMemoComponent,
      LazyComponent,
      FundamentalComponent,
      ScopeComponent,
    } from 'shared/ReactWorkTags';
    import getComponentName from 'shared/getComponentName';

    import {isDevToolsPresent} from './ReactFiberDevToolsHook';
    import {
      resolveClassForHotReloading,
      resolveFunctionForHotReloading,
      resolveForwardRefForHotReloading,
    } from './ReactFiberHotReloading';
    import {NoWork} from './ReactFiberExpirationTime';
    import {
      NoMode,
      ConcurrentMode,
      ProfileMode,
      StrictMode,
      BatchedMode,
    } from './ReactTypeOfMode';
    import {
      REACT_FORWARD_REF_TYPE,
      REACT_FRAGMENT_TYPE,
      REACT_STRICT_MODE_TYPE,
      REACT_PROFILER_TYPE,
      REACT_PROVIDER_TYPE,
      REACT_CONTEXT_TYPE,
      REACT_CONCURRENT_MODE_TYPE,
      REACT_SUSPENSE_TYPE,
      REACT_SUSPENSE_LIST_TYPE,
      REACT_MEMO_TYPE,
      REACT_LAZY_TYPE,
      REACT_FUNDAMENTAL_TYPE,
      REACT_SCOPE_TYPE,
    } from 'shared/ReactSymbols';
}

In [ ]:
{
    type Dependencies = {
        expirationTime:ExpirationTime,
        firstContext:ContextDependency<mixed> | null,
        responders:Map<ReactEventResponder<any,any>, ReactEventResponderInstance<any, any>> | null
    }
}

In [ ]:
    //a fiber is work on a component that needs to be done or was done. there can be more than one per component.

    //these first fields are conceptually members of an Instance. This used to be split into a separate type
    //and intersected with the other fiber fields, but until fixes its intersection bugs, we've merged them into a
    // single type.

    // an instance is shared between all versions of a component. we can easily break this out into a separate object to avoid
    // copying so much to the alternate versions of the tree. we put this on a single object for now to minimize the number of
    // objects created during the initial render.
    type Fiber = {
        //Tag identifying the type of fiber
        tag:WorkTag,
        //Unique identifier of this child
        key:null|string,
        //the value of eleent.type which is used to preserve the identity during reconciliation of this child.
        elementType:any,
        //the resolved function/class associated with this fiber.
        type:any,
        //the  local state associated with this fiber.
        stateNode:any,
        
        // conceptual aliases
        // parent: instane -> return the parent happens to be the same as the
        // return fiber since we'vw merged the fiber and instance.
        
        // remaining fields belong to Fiber
        
        // the fiber to return to after finishing processing this one.
        // this is effectively the parent, but there can be multiple parents (tow)
        // so this is only the parent of the thing we're currently processing.
        // it is conceptually the same as the return address of a stack frame.
        return:Fiber|null,
        
        // singly linked list tree structure.
        child:Fiber|null,
        sibling:Fiber|null,
        index:number,
        
        //the ref last used to attach this node
        //i'll avoid adding an owner field for prod and model that as functions.
        ref:null | (((handle: mixed) => void) & {_stringRef:?string}) | RefObject,
        
        //input is the data coming into process this fiber.Arguments.props.
        pendingProps:any, //this type will be more specific once we overload the tag.
        memoizedProps:any, //the props used to create the output
        
        // a queue of state updates and callbacks
        updateQueue:UpdateQueue<any> | null,
        
        //the state used to create the output
        memoizedState:any,
        
        //dependencies (contexts, events) for this fiber, if it has any
        dependencies:Dependencies | null,
        
        //bitfield that describes properties about the fiber and its subtree. e.g.
        //the concurrentMode flag indicates wheher the subtree should be async-by-default.
        //when a fiber is created, it inherits the mode of its parent.
        // additional flags can be set at creation time, but after that the value should remain
        // unchanged throughout the fiber's lifetime, particularly before its child fibers are created.
        mode:TypeOfMode,
        
        //Effect
        effectTag:SideEffectTag,
        
        //singly linked list fast path to the next fiber with side-effects.
        nextEffect:Fiber | null,
        
        //the first and last fiber with side-effect within this subtree. this allows us to reuse  a slice of the linked list
        //when we reuse the work done within this fiber.
        firstEffect:Fiber | null,
        lastEffect:Fiber | null,
        
        //represents a time in the future by which this work should be completed.
        //does not include work found in its subtree.
        expirationTime:ExpirationTime,
        
        // this is used to quickly determine if a subtree has no pending changes.
        childExpirationTime:ExpirationTime,
        
        //this is a pooled version of a Fiber. every fiber that gets updated will 
        //eventually have a pair. there are cases when we can clean up pairs to save
        //memory if we need to.
        alternate:Fiber | null,
        
        //time spent rendering this fiber and its descpendants for the current update.
        //this tells us how well the tree makes use of sCU for memoization.
        //it is reset to 0 each time we renderand only updated when we don't bailout.
        //this field is only set when the enableProfilerTimer flag is enabled.
        actualDuration?: number,
        
        //if the fiber is currently active in the 'render' phase,
        //this marks the time at which the work began.
        //this field is only set when the enableProfilerTimer flag is enabled.
        actualStartTime?:number,
        
        //duration fo the most recent render time for this fiber.
        //this value is not updated when we bailout for memoization purposes.
        //this field is only set when the enableProfilerTimer flag is enabled.
        selfBaseDuration?:number,
        
        //sum of base times for all descendants of this fiber.
        // this value bubbles up during the "complete" phase.
        // this field is only set when the enableProfilerTimer flag is enabled.
        treeBaseDuration?:number,
        
        //conceptual aliases
        //workInProgree: fiber => alternate the alternate used for reuse happens
        // to be the same as work in progree.
        //__DEV__ only
        _debugID?:number,
        _debugSource?:Source | null,
        _debugOwner?:Fiber | null,
        _debugIsCurrentlyTiming?:boolean,
        _debugNeedsRemount?:boolean,
        
        // used to verify that the order of hooks does not change between renders.
        _debugHookTypes?:Array<HookType> | null
    }
}

Fiber 的整个数据结构

In [ ]:
{
    let debugCounter = 1;
}

?? debug 设置 ??

In [ ]:
{
    function FiberNode(
        tag:WorkTag,
        pendingProps:mixed,
        key:null | string,
        mode:TypeOfMode
    ){
        //instance
        this.tag = tag;
        this.key = key;
        this.elementType = null;
        this.type = null;
        this.stateNode = null;
            
        //fiber
        this.return = null;
        this.child = null;
        this.sibling = null;
        this.index = 0;
            
        this.ref = null;
            
        this.pendingProps = pendingProps;
        this.memoizedProps = null;
        this.undateQueue = null;
        this.memoizedState = null;
        this.dependencies = null;
            
        this.mode = mode;
        
        //effects
        this.effectTag= NoEffect;
        this.nextEffect = null;
            
        this.firstEffect = null;
        this.lastEffect = null;
            
        this.expirationTime = NoWork;
        this.childExpirationTime = NoWork;
            
        this.alternate = null;
            
        if(enableProfilerTier){
            //note: the following is done to avoid a v8 performace cliff.
            
            // initializing the fields below o smis and later updating them with
            // double values will cause Fibers to end up having separate shapes.
            // this behavior/bug has something to do with object.preventExtension().
            //fortunately this only impacts DEV builds.
            //upfortunately it makes React unusably slow for some applications.
            //to work around this, initialize the fields below with doubles.
            //
            // Learn more about this here:
            // https://github.com/facebook/react/issues/14365
            // https://bugs.chromium.org/p/v8/issues/detail?id=8538
            this.actualDuration = Number.NaN;
            this.actualStartTime = Number.NaN;
            this.selfBaseDuration = Number.NaN;
            this.treeBaseDuration = Number.NaN;
            
            //it's okay to replac4e the initial doubles with smis after initialization.
            //this won't trigger the performance cliff mentioned above,
            // and it simplifies oher profiler code (includeing DevTools).
            this.actualDuration = 0;
            this.actualStartTime = -1;
            this.selfBaseDuration = 0;
            this.treeBaseDuration = 0;
        }
            
        // this is normally DEV-only except www when it adds listeners.
        // todo: remove the user timing integration in favor of root events.
        if(enableUserTimingAPI){
            this._debugID = debugCounter++;
            this._debugIsCurrentlyTiming = false;
        }
            
        
    }
    
    //this is a constructor function, ranther than a POJO constructor, still
    //please ensure we do the following:
    //1) nobody should add any instance methods on this. instance methods can be 
    //   more difficult to predict when they get optimized and they are almost
    //   never inlined properly in static compilers.
    //2) nobody should rely on `instanceof Fiber` for type testing. we should
    //   always know when it is a fiber.
    //3) we might want to experiment with using numberic keys since the are easier
    //   to optimize in a non-JIT environment.
    //4) we can easily go from a constructor to a createFiber object literal if htat
    //   is faster.
    //5) it should be easy to port this to a c struct and keep a c impleentation
    //    compatible
    const createFiber = function(
        tag:WorkTag,
        pendingProps:mixed,
        key:null | string,
        mode:TypeOfMode
    ):Fiber {
        return new FiberNode(tag,pendingProps,key,mode);
    };
}

In [ ]:
{
    function shouldConstruct(Component:Function){
        const prototype = Component.prototype;
        return !!(prototype && prototype.isReactComponent);
    }
    
    function isSimpleFunctionComponent(type:any){
        return (
            typeof type === 'function' &&
            !shouldConstruct(type) &&
            type.defaultProps === undefined
        )
    }
    
    function resolveLazyComponentTag(Component:Function):WorkTag{
        if(typeof Component === 'function'){
            return shouldConstruct(Component)?ClassComponent:FunctionComponent;
        }else if(Component !== undefined && Component != null){
            const $$typeof == Component.$$typeof;
            if($$typeof === REACT_FORWARD_REF_TYPE){
                return ForwardRef;
            }
            if($$typeof === REACT_MEMO_TYPE){
                return MemoComponent;
            }
        }
        return IndeterminateComponent;
    }
}

In [ ]:
{
    //this is used to create an alternate fiber to do work on.
    function createWorkInProgress(
        current:Fiber,
        pendingProps:any,
        expirationTime:ExpirationTime
    ):Fiber {
        let workInProgress = current.alternate;
        if(workInProgress === null){
            // we use a double buffering pooling technique because we know that we'll
            // only ever need a most two version of a tree, we pool the 'other' unused
            // node that we're free to reuse. this is lazily created to avoid allocationg
            // extra objects for things that are never updated. it also allow us to
            // reclaim the extra memory if needed.
            workInProgress = createFiber(
                current.tag,
                pendingProps,
                current.key,
                current.mode
            );
            workInProgress.elementType = current.elementType;
            workInProgress.type = current.type;
            workInProgress.stateNode = current.stateNode;
            
            workInProgress.alternate = current;
            current.alternate = workInProgress;
        }else{
            workInProgress.pendingProps = pendingProps;
            
            // we already have an alternate.
            // Reset the effect tag.
            workInProgress.effectTag = NoEffect;
            
            // the effect list is no longer valid.
            workInProgress.nextEffect = null;
            workInProgress.firstEffect = null;
            workInProgress.lastEffect = null;
            
            if(enableProfilerTimer){
                // we intentionally reset, rather the copy ,actualDuration & actualStartTime.
                // this prevents time fro enlessly accumulating in new commits.
                // this has the downside of resetting values for different priority renders,
                // but works for yielding(the common case) and should support resuming.
                workInProgress.actualDuration = 0;
                workInProgress.actualStartTime = -1;
            }
        }
        
        workInProgress.childExpirationTime = current.childExpirationTime;
        workInProgress.expirationTime = current.expirationTime;
            
        workInProgress.child = current.child;
        workInProgress.memoizedProps = current.memoizedProps;
        workInProgress.memoizedState = current.memoizedState;
        workInProgress.updateQueue = current.updateQueue;
            
        // clone the dependencies object. this is mutated during the render phase,
        // so it cannot be shared with the current fiber.
        const currentDependencies = current.dependencies;
        workInProgress.dependencies = 
            currentDependencies === null
                ? null
                : {
                    expirationTime:currentDependencies.expirationTime,
                    firstContext:currentDependencies.firstContext,
                    responders:currentDependencies.responders
                };
        
        // these will be overridden during the paren's reconciliation
        workInProgress.sibling = current.sibling;
        workInProgress.index = current.index;
        workInProgress.ref = current.ref;
            
        if(enableProfilerTimer){
            workInProgress.selfBaseDuration = current.selfBaseDuration;
            workInProgress.treeBaseDuration = current.treeBaseDuration;
        }
        
        reurn workInProgress;
    }
}

`createWorkInProgress`

In [ ]:
{
    //used to reuse a fiber for a second pass.
    function resetWorkInProgress(
        workInProgress:Fiber,
        renderExpirationime:ExpirationTime
    ){
        //this resets the fiber to what createFiber or createWorkInProgress would
        //have set the values to before during the first pass. ideally this wouldn't
        //be necessary but unfortunately many code paths reads from the workInProgress
        // when they should be reading from current and writeing to workInProgress.
        
        //we assume pendingProps, index, key, ref, return are still untoched to
        //avoid doing another reconciliation.
            
        //reset the effect tag but keep any placement tags, since that's something
        // that child fiber is setting, not the reconciliation.
        workInProgress.effectTag &= Placement;
            
        // the effect list is no longer valid.
        workInProgress.nextEffect = null;
        workInProgress.firstEffect = null;
        workInProgress.lasEffect = null;
            
        let current = workInProgress.alternate;
        if(current === null) {
            // Reset to createFiber's initial values.
            workInProgress.childExpirationTime = NoWork;
            workInProgress.expirationTime = renderExpirationTime;
            
            workInProgress.child = null;
            workInProgress.memoizedProps = null;
            workInProgress.memoizedSate = null;
            workInProgress.updateQueue = null;
            
            workInProgress.dependencies = null;
            
            if(enableProfilerTimer){
                //Note: we don't reset the actualTime counts. it's useful to accumulate
                //actual time across multiple render passes.
                workInProgress.selfBaseDuration = 0;
                workInProgress.treeBaseDuration = 0;
            }
        }else{
            //reset to the cloned values that createWorkInProgress would've.
            workInProgress.childExpirationTime = current.childExpirationTime;
            workInProgress.expirationTime = current.expirationTime;
            
            workInProgress.child = current.child;
            workInProgress.memoizedProps = current.memoizedProps;
            workInProgress.memoizedSate = current.memoizedSate;
            workInProgress.updateQueue = current.updateQueue;
            
            //clone the dependencies object. this is mutated during the render phase, so
            // it annot be shared with the current fiber.
            const currentDependencies = current.dependencies;
            workInProgress.dependencies =
                currentDependencies === null
                    ? null
                    : {
                        expirationTime:currentDependencies.expirationTime,
                        firstContext:currentDependencies.firstContext,
                        responders:currentDependencies.responders
                    };
            
            if(enableProfilerTimer){
                //mote :we don't reset the actualTime counts. it's useful to acumulate
                // actual time across multiple render passes.
                workInProgress.selfBaseDuration = current.selfBaseDuration;
                workInProgress.treeBaseDuration = current.treeBAseDuration;
            }
        }
        
        return workInProgress;        
    }
}

`resetWorkInProgress`

In [ ]:
{
    function createHostRootFiber(tag:RootTag):Fiber{
        let mode;
        if(tag === ConcurrentRoot){
            mode = ConcurrentMode | BatchedMode | StricuMode;
        }else if(tag === BatchedRoot){
            mode = BatchedMode | StrictMode;
        }else{
            mode = NoMode;
        }
        
        if(enableProfilerTimer && isDevToolsPresent){
            //Always collect profile timings when DevTools are present.
            //thsi enables DevTools to start capturing timing at any point-Without
            //some nodes in the tree having empty base times.
            mode |= ProfileMode;
        }
        
        return createFiber(HostRoot,null,null,mode);
    }
    
    function createFiberFromTypeAndProps(
        type:any, //React$ElementType
        key:null | string,
        pendingProps:any,
        owner:null | Fiber,
        mode:TypeOfMode,
        expirationTime:ExpirationTime
    ):Fiber {
            let fiber;
            
            let fiberTag = IndeterminateComponent;
            //the resolved type is set if we know what the final type will be.it's not lazy.
            let resolvedType = type;
            if(typeof type === 'function'){
                if(shouldConstruct(type)){
                    fiberTag = ClassComponent;
                }
            }else if(tpyeof type === 'string'){
                fiberTag = HostComponent;
            }else{
                getTag:switch(type){
                    case REACT_FRAGMENT_TYPE:
                        return createFiberFromFragment(
                            pendingProps.children,
                            mode,
                            expirationTime,
                            key
                        )
                    case REACT_CONCURRENT_MODE_TYPE:
                        fiberTag = Mode;
                        mode |= ConcurrentMode | BatchedMode | StrictMode;
                        break;
                    case REACT_STRICT_MODE_TYPE:
                        fiberTag = Mode;
                        mode |= StrictMode;
                        break;
                    case REACT_PROFILER_TYPE:
                        return createFiberFromProfiler(pendingProps,mode,expirationTime,key);
                    case REACT_SUSPENSE_TYPE:
                        return createFiberFromSuspense(pendingProps,mode,expirationTime,key);
                    case REACT_SUSPENSE_LIST_TYPE:
                        return createFiberFromSuspenseList(
                            pendngProps,
                            mode,
                            expirationTime,
                            key
                        );
                    default:{
                        if(typeof type === 'object' && type!==null){
                            switch(type.$$typeof){
                                case REACT_PROVIDER_TYPE:
                                    fiberTag = ContextProvider;
                                    break getTag;
                                case REACT_CONTEXT_TYPE:
                                    //this is a consumer
                                    fiberTag = ContextConsumer;
                                    break getTag;
                                case REACT_FORWARD_REF_TYPE:
                                    fiberTag = ForwardRef;
                                    break getTag;
                                case REACT_MEMO_TYPE:
                                    fiberTag = MemoComponent;
                                    break getTag;
                                case REACT_LAZY_TYPE:
                                    fiberTag = LazyComponent;
                                    resolvedType = null;
                                    break getTag;
                                case REACT_FUNDAMENTAL_TYPE:
                                    if(enableFundamentalAPI){
                                        return createFiberFromFundamental(
                                            type,
                                            pendingProps,
                                            mode,
                                            expirationTime,
                                            key
                                        )
                                    }
                                    break;
                                case REACT_SCOPE_TYPE:
                                    if(enableScopeAPI){
                                        return createFiberFromScope(
                                            type,
                                            pendingProps,
                                            mode,
                                            expirationTime,
                                            key
                                        );
                                    }
                            }
                        }
                        let info = '';
                        invariant(
                          false,
                          'Element type is invalid: expected a string (for built-in ' +
                            'components) or a class/function (for composite components) ' +
                            'but got: %s.%s',
                          type == null ? type : typeof type,
                          info,
                        );
                    }
                }
            }
            
            fiber = createFiber(fiberTag,pendingProps,key,mode);
            fiber.elementType = type;
            fiber.type = resolvedType;
            fiber.expirationTie = expirationTime;
            
            return fiber;
        }
    
    function createFiberFromElement(
        element:ReatElement,
        mode:TypeOfMode,
        expirationTime:ExpirationTime
    ):Fiber {
            let owner = null;
            const type = element.type;
            const key = element.key;
            const pendingProps = element.props;
            const fiber = createFiberFromTypeAdnProps(
                type,
                key,
                pendingProps,
                owner,
                mode,
                expirationTime
            );
            return fiber;
        }
    
    function createFiberFromFragment(
        elements:ReactFragment,
        mode: TypeOfMode,
        expirationTime:ExpirationTime,
        key: null | string
    ):Fiber{
            const fiber = createFiber(Fragment,elements,key,mode);
            fiber.expirationTime = expirationTime;
            return fiber;
        }
    
    function createFiberFromFundamental(
        fundamentalComponent:ReactFundamentalComponent<any,any>,
        pendingProps:any,
        mode:TypeOfMode,
        expirationTime:ExpirationTime,
        key:null | string
    ):Fiber{
        const fiber = createFiber(FundamentalComponent,pendingProps,key,mode);
        fiber.elementTpe = fundamentalComponent;
        fiber.type = fundamentalComponent;
        fiber.expirationTime = expirationTime;
        return fiber;
    }
    
    function createFiberFromScope(
        scope:ReactScope,
        pendingProps:any,
        mode:TypeOfMode,
        expirationTime:ExpirationTime,
        key:null | string
    ):Fiber{
            const fiber = createFiber(ScopeComponent,pendingProps,key,mode);
            fiber.type = scope;
            fiber.elementType = scope;
            fiber.expirationTime = expirationTime;
            return fiber;
        }
    
    function createFiberFromProfiler(
        pendingProps:any,
        mode:TypeOfMode,
        expirationTime:ExpirationTime,
        key:null | string
    ):Fiber{
            const fiber = createFiber(Profiler,pendingProps,key,mode|ProfileMode);
            //todo the profiler fiber shouldn't hava a type,it has a tag.
            fiber.elementType = REACT_PROFILER_TYPE;
            fiber.type = REACT_PROFILER_TYPE;
            fiber.expirationTime = expirationTime;
            
            return fiber;
        }
    
    function createFiberFromSuspense(
        pendingProps:any,
        mode:TypeOfMode,
        expirationTime:ExpirationTime,
        key:null |string
    ):Fiber{
            const fiber = createFiber(SuspenseComponent, pendingProps,key,mode);
            
            //todo :the suspenseComponent fiber shouldn't hava a type . it has a tag.
            // this needs to be fixed in getComponentName se that is relies on the tag
            // instead.
            fiber.type = REACT_SUSPENSE_TYPE;
            fiber.elementType = REACT_SUSPENSE_TYPE;
            
            fiber.expirationTime = expirationTime;
            return fiber;
        }
    
    function createFiberFroSuspenseList(
        pendingProps:any,
        mode:TypeOfMode,
        expirationTime:ExpirationTime,
        key:null | string
    ):Fiber{
            const fiber = createFiber(SuspenseListComponent,pendingProps,key,mode);
            fiber.elementType = REACT_SUSPENSE_LIST_TYPE;
            fiber.expirationTime = expirationTime;
            return fiber;
        }
    
    function createFiberFromText(
        content:string,
        mode:TypeOfMode,
        expirationTime:ExprationTime
    ):Fiber{
            const fiber = reateFiber(HostText,content,null,mode);
            fiber.expirationTime = expirationTime;
            return fiber;
        }
    
    function createFiberFromHostInstanceForDeletion():Fiber{
        const fiber = createFiber(HostComponent,null,null,NoMode);
        //todo :these should not need a type
        fiber.elementType ='DELETED';
        fiber.type ='DELETED';
        return fiber;
    }
    
    function createFiberFromDehydratedFragment(
        dehydratedNode:SuspenseInstance
    ):Fiber{
            const fiber = createFiber(DehydratedFragment,null,null,NoMode);
            fiber.stateNode = dehydratedNode;
            return fiber;
        }
    
    function createFiberFromPortal(
        portal:ReactPortal,
        mode:TypeOfMode,
        expirationTime:ExpirationTime
    ):Fiber{
            const pendingProps = portal.children!==null?portal.children:[];
            const fiber = createFiber(HostPortal,pendingProps,portal.key,mode);
            fiber.expirationTime = expirationTime;
            fiber.stateNode = {
                containerInfo:portal.containerInfo,
                pendingChildren:null,//used by persistent updates
                implementation:portal.implementation
            };
            return fiber;
        }
    
    //used for stashing WIP properties to replay failed work in DEV.
    function assignFiberPropertiesInDEV(
        target:Fiber|null,
        source:Fiber
    ):Fiber{
            if(target === null){
                //this fiber's initial properties will always be overwritten.
                //we only use a fiber to ensure the same hidden class so DEV isn't slow;
                target= createFiber(IndeterminateComponent,null,null,NoMode);
            }
            
            //this is inientionally writen as a list of all properties.
            //we tried to use object.assign() instead but this is called in
            //this hottest path, and object.assign() was too slow:
            //https://github.com/facebook/react/issues/12502
            //this code is DEV-only so size is not concern.
              target.tag = source.tag;
              target.key = source.key;
              target.elementType = source.elementType;
              target.type = source.type;
              target.stateNode = source.stateNode;
              target.return = source.return;
              target.child = source.child;
              target.sibling = source.sibling;
              target.index = source.index;
              target.ref = source.ref;
              target.pendingProps = source.pendingProps;
              target.memoizedProps = source.memoizedProps;
              target.updateQueue = source.updateQueue;
              target.memoizedState = source.memoizedState;
              target.dependencies = source.dependencies;
              target.mode = source.mode;
              target.effectTag = source.effectTag;
              target.nextEffect = source.nextEffect;
              target.firstEffect = source.firstEffect;
              target.lastEffect = source.lastEffect;
              target.expirationTime = source.expirationTime;
              target.childExpirationTime = source.childExpirationTime;
              target.alternate = source.alternate;
              if (enableProfilerTimer) {
                target.actualDuration = source.actualDuration;
                target.actualStartTime = source.actualStartTime;
                target.selfBaseDuration = source.selfBaseDuration;
                target.treeBaseDuration = source.treeBaseDuration;
              }
              target._debugID = source._debugID;
              target._debugSource = source._debugSource;
              target._debugOwner = source._debugOwner;
              target._debugIsCurrentlyTiming = source._debugIsCurrentlyTiming;
              target._debugNeedsRemount = source._debugNeedsRemount;
              target._debugHookTypes = source._debugHookTypes;
              return target;
        }
}

In [ ]:
{
    import type {ReactElement, Source} from 'shared/ReactElementType';
    import type {
      ReactFragment,
      ReactPortal,
      RefObject,
      ReactEventResponder,
      ReactEventResponderInstance,
      ReactFundamentalComponent,
      ReactScope,
    } from 'shared/ReactTypes';
    import type {RootTag} from 'shared/ReactRootTags';
    import type {WorkTag} from 'shared/ReactWorkTags';
    import type {TypeOfMode} from './ReactTypeOfMode';
    import type {SideEffectTag} from 'shared/ReactSideEffectTags';
    import type {ExpirationTime} from './ReactFiberExpirationTime';
    import type {UpdateQueue} from './ReactUpdateQueue';
    import type {ContextDependency} from './ReactFiberNewContext';
    import type {HookType} from './ReactFiberHooks';
    import type {SuspenseInstance} from './ReactFiberHostConfig';
    
}

import 引入代码作用

**ReactElementType.js** 

```Source --> [fileName,lineNumber]```
```ReactElement --> [$$typeof,type,key,ref,props,_owner, //DEV _store:{validated},_self,_shadownChildren,_source]```

`Source` 是用于出错时判断文件，行数。
`ReactElement` 是`React` 生成 `Element` 的通用数据。

**ReactTypes.js**

定义react相关元素的结构。

**HookType**
```
export type HookType =
  | 'useState'
  | 'useReducer'
  | 'useContext'
  | 'useRef'
  | 'useEffect'
  | 'useLayoutEffect'
  | 'useCallback'
  | 'useMemo'
  | 'useImperativeHandle'
  | 'useDebugValue'
  | 'useResponder';
```
... 同理后续文件


In [ ]:
{
    import invariant from 'shared/invariant';
    import warningWithoutStack from 'shared/warningWithoutStack';
    import {
      enableProfilerTimer,
      enableFundamentalAPI,
      enableUserTimingAPI,
      enableScopeAPI,
    } from 'shared/ReactFeatureFlags';
    import {NoEffect, Placement} from 'shared/ReactSideEffectTags';
    import {ConcurrentRoot, BatchedRoot} from 'shared/ReactRootTags';
    import {
      IndeterminateComponent,
      ClassComponent,
      HostRoot,
      HostComponent,
      HostText,
      HostPortal,
      ForwardRef,
      Fragment,
      Mode,
      ContextProvider,
      ContextConsumer,
      Profiler,
      SuspenseComponent,
      SuspenseListComponent,
      DehydratedFragment,
      FunctionComponent,
      MemoComponent,
      SimpleMemoComponent,
      LazyComponent,
      FundamentalComponent,
      ScopeComponent,
    } from 'shared/ReactWorkTags';
    import getComponentName from 'shared/getComponentName';

    import {isDevToolsPresent} from './ReactFiberDevToolsHook';
    import {
      resolveClassForHotReloading,
      resolveFunctionForHotReloading,
      resolveForwardRefForHotReloading,
    } from './ReactFiberHotReloading';
    import {NoWork} from './ReactFiberExpirationTime';
    import {
      NoMode,
      ConcurrentMode,
      ProfileMode,
      StrictMode,
      BatchedMode,
    } from './ReactTypeOfMode';
    import {
      REACT_FORWARD_REF_TYPE,
      REACT_FRAGMENT_TYPE,
      REACT_STRICT_MODE_TYPE,
      REACT_PROFILER_TYPE,
      REACT_PROVIDER_TYPE,
      REACT_CONTEXT_TYPE,
      REACT_CONCURRENT_MODE_TYPE,
      REACT_SUSPENSE_TYPE,
      REACT_SUSPENSE_LIST_TYPE,
      REACT_MEMO_TYPE,
      REACT_LAZY_TYPE,
      REACT_FUNDAMENTAL_TYPE,
      REACT_SCOPE_TYPE,
    } from 'shared/ReactSymbols';
}

In [ ]:
{
    type Dependencies = {
        expirationTime:ExpirationTime,
        firstContext:ContextDependency<mixed> | null,
        responders:Map<ReactEventResponder<any,any>, ReactEventResponderInstance<any, any>> | null
    }
}

In [ ]:
    //a fiber is work on a component that needs to be done or was done. there can be more than one per component.

    //these first fields are conceptually members of an Instance. This used to be split into a separate type
    //and intersected with the other fiber fields, but until fixes its intersection bugs, we've merged them into a
    // single type.

    // an instance is shared between all versions of a component. we can easily break this out into a separate object to avoid
    // copying so much to the alternate versions of the tree. we put this on a single object for now to minimize the number of
    // objects created during the initial render.
    type Fiber = {
        //Tag identifying the type of fiber
        tag:WorkTag,
        //Unique identifier of this child
        key:null|string,
        //the value of eleent.type which is used to preserve the identity during reconciliation of this child.
        elementType:any,
        //the resolved function/class associated with this fiber.
        type:any,
        //the  local state associated with this fiber.
        stateNode:any,
        
        // conceptual aliases
        // parent: instane -> return the parent happens to be the same as the
        // return fiber since we'vw merged the fiber and instance.
        
        // remaining fields belong to Fiber
        
        // the fiber to return to after finishing processing this one.
        // this is effectively the parent, but there can be multiple parents (tow)
        // so this is only the parent of the thing we're currently processing.
        // it is conceptually the same as the return address of a stack frame.
        return:Fiber|null,
        
        // singly linked list tree structure.
        child:Fiber|null,
        sibling:Fiber|null,
        index:number,
        
        //the ref last used to attach this node
        //i'll avoid adding an owner field for prod and model that as functions.
        ref:null | (((handle: mixed) => void) & {_stringRef:?string}) | RefObject,
        
        //input is the data coming into process this fiber.Arguments.props.
        pendingProps:any, //this type will be more specific once we overload the tag.
        memoizedProps:any, //the props used to create the output
        
        // a queue of state updates and callbacks
        updateQueue:UpdateQueue<any> | null,
        
        //the state used to create the output
        memoizedState:any,
        
        //dependencies (contexts, events) for this fiber, if it has any
        dependencies:Dependencies | null,
        
        //bitfield that describes properties about the fiber and its subtree. e.g.
        //the concurrentMode flag indicates wheher the subtree should be async-by-default.
        //when a fiber is created, it inherits the mode of its parent.
        // additional flags can be set at creation time, but after that the value should remain
        // unchanged throughout the fiber's lifetime, particularly before its child fibers are created.
        mode:TypeOfMode,
        
        //Effect
        effectTag:SideEffectTag,
        
        //singly linked list fast path to the next fiber with side-effects.
        nextEffect:Fiber | null,
        
        //the first and last fiber with side-effect within this subtree. this allows us to reuse  a slice of the linked list
        //when we reuse the work done within this fiber.
        firstEffect:Fiber | null,
        lastEffect:Fiber | null,
        
        //represents a time in the future by which this work should be completed.
        //does not include work found in its subtree.
        expirationTime:ExpirationTime,
        
        // this is used to quickly determine if a subtree has no pending changes.
        childExpirationTime:ExpirationTime,
        
        //this is a pooled version of a Fiber. every fiber that gets updated will 
        //eventually have a pair. there are cases when we can clean up pairs to save
        //memory if we need to.
        alternate:Fiber | null,
        
        //time spent rendering this fiber and its descpendants for the current update.
        //this tells us how well the tree makes use of sCU for memoization.
        //it is reset to 0 each time we renderand only updated when we don't bailout.
        //this field is only set when the enableProfilerTimer flag is enabled.
        actualDuration?: number,
        
        //if the fiber is currently active in the 'render' phase,
        //this marks the time at which the work began.
        //this field is only set when the enableProfilerTimer flag is enabled.
        actualStartTime?:number,
        
        //duration fo the most recent render time for this fiber.
        //this value is not updated when we bailout for memoization purposes.
        //this field is only set when the enableProfilerTimer flag is enabled.
        selfBaseDuration?:number,
        
        //sum of base times for all descendants of this fiber.
        // this value bubbles up during the "complete" phase.
        // this field is only set when the enableProfilerTimer flag is enabled.
        treeBaseDuration?:number,
        
        //conceptual aliases
        //workInProgree: fiber => alternate the alternate used for reuse happens
        // to be the same as work in progree.
        //__DEV__ only
        _debugID?:number,
        _debugSource?:Source | null,
        _debugOwner?:Fiber | null,
        _debugIsCurrentlyTiming?:boolean,
        _debugNeedsRemount?:boolean,
        
        // used to verify that the order of hooks does not change between renders.
        _debugHookTypes?:Array<HookType> | null
    }
}

Fiber 的整个数据结构

In [ ]:
{
    let debugCounter = 1;
}

?? debug 设置 ??

In [ ]:
{
    function FiberNode(
        tag:WorkTag,
        pendingProps:mixed,
        key:null | string,
        mode:TypeOfMode
    ){
        //instance
        this.tag = tag;
        this.key = key;
        this.elementType = null;
        this.type = null;
        this.stateNode = null;
            
        //fiber
        this.return = null;
        this.child = null;
        this.sibling = null;
        this.index = 0;
            
        this.ref = null;
            
        this.pendingProps = pendingProps;
        this.memoizedProps = null;
        this.undateQueue = null;
        this.memoizedState = null;
        this.dependencies = null;
            
        this.mode = mode;
        
        //effects
        this.effectTag= NoEffect;
        this.nextEffect = null;
            
        this.firstEffect = null;
        this.lastEffect = null;
            
        this.expirationTime = NoWork;
        this.childExpirationTime = NoWork;
            
        this.alternate = null;
            
        if(enableProfilerTier){
            //note: the following is done to avoid a v8 performace cliff.
            
            // initializing the fields below o smis and later updating them with
            // double values will cause Fibers to end up having separate shapes.
            // this behavior/bug has something to do with object.preventExtension().
            //fortunately this only impacts DEV builds.
            //upfortunately it makes React unusably slow for some applications.
            //to work around this, initialize the fields below with doubles.
            //
            // Learn more about this here:
            // https://github.com/facebook/react/issues/14365
            // https://bugs.chromium.org/p/v8/issues/detail?id=8538
            this.actualDuration = Number.NaN;
            this.actualStartTime = Number.NaN;
            this.selfBaseDuration = Number.NaN;
            this.treeBaseDuration = Number.NaN;
            
            //it's okay to replac4e the initial doubles with smis after initialization.
            //this won't trigger the performance cliff mentioned above,
            // and it simplifies oher profiler code (includeing DevTools).
            this.actualDuration = 0;
            this.actualStartTime = -1;
            this.selfBaseDuration = 0;
            this.treeBaseDuration = 0;
        }
            
        // this is normally DEV-only except www when it adds listeners.
        // todo: remove the user timing integration in favor of root events.
        if(enableUserTimingAPI){
            this._debugID = debugCounter++;
            this._debugIsCurrentlyTiming = false;
        }
            
        
    }
    
    //this is a constructor function, ranther than a POJO constructor, still
    //please ensure we do the following:
    //1) nobody should add any instance methods on this. instance methods can be 
    //   more difficult to predict when they get optimized and they are almost
    //   never inlined properly in static compilers.
    //2) nobody should rely on `instanceof Fiber` for type testing. we should
    //   always know when it is a fiber.
    //3) we might want to experiment with using numberic keys since the are easier
    //   to optimize in a non-JIT environment.
    //4) we can easily go from a constructor to a createFiber object literal if htat
    //   is faster.
    //5) it should be easy to port this to a c struct and keep a c impleentation
    //    compatible
    const createFiber = function(
        tag:WorkTag,
        pendingProps:mixed,
        key:null | string,
        mode:TypeOfMode
    ):Fiber {
        return new FiberNode(tag,pendingProps,key,mode);
    };
}

In [ ]:
{
    function shouldConstruct(Component:Function){
        const prototype = Component.prototype;
        return !!(prototype && prototype.isReactComponent);
    }
    
    function isSimpleFunctionComponent(type:any){
        return (
            typeof type === 'function' &&
            !shouldConstruct(type) &&
            type.defaultProps === undefined
        )
    }
    
    function resolveLazyComponentTag(Component:Function):WorkTag{
        if(typeof Component === 'function'){
            return shouldConstruct(Component)?ClassComponent:FunctionComponent;
        }else if(Component !== undefined && Component != null){
            const $$typeof == Component.$$typeof;
            if($$typeof === REACT_FORWARD_REF_TYPE){
                return ForwardRef;
            }
            if($$typeof === REACT_MEMO_TYPE){
                return MemoComponent;
            }
        }
        return IndeterminateComponent;
    }
}

In [ ]:
{
    //this is used to create an alternate fiber to do work on.
    function createWorkInProgress(
        current:Fiber,
        pendingProps:any,
        expirationTime:ExpirationTime
    ):Fiber {
        let workInProgress = current.alternate;
        if(workInProgress === null){
            // we use a double buffering pooling technique because we know that we'll
            // only ever need a most two version of a tree, we pool the 'other' unused
            // node that we're free to reuse. this is lazily created to avoid allocationg
            // extra objects for things that are never updated. it also allow us to
            // reclaim the extra memory if needed.
            workInProgress = createFiber(
                current.tag,
                pendingProps,
                current.key,
                current.mode
            );
            workInProgress.elementType = current.elementType;
            workInProgress.type = current.type;
            workInProgress.stateNode = current.stateNode;
            
            workInProgress.alternate = current;
            current.alternate = workInProgress;
        }else{
            workInProgress.pendingProps = pendingProps;
            
            // we already have an alternate.
            // Reset the effect tag.
            workInProgress.effectTag = NoEffect;
            
            // the effect list is no longer valid.
            workInProgress.nextEffect = null;
            workInProgress.firstEffect = null;
            workInProgress.lastEffect = null;
            
            if(enableProfilerTimer){
                // we intentionally reset, rather the copy ,actualDuration & actualStartTime.
                // this prevents time fro enlessly accumulating in new commits.
                // this has the downside of resetting values for different priority renders,
                // but works for yielding(the common case) and should support resuming.
                workInProgress.actualDuration = 0;
                workInProgress.actualStartTime = -1;
            }
        }
        
        workInProgress.childExpirationTime = current.childExpirationTime;
        workInProgress.expirationTime = current.expirationTime;
            
        workInProgress.child = current.child;
        workInProgress.memoizedProps = current.memoizedProps;
        workInProgress.memoizedState = current.memoizedState;
        workInProgress.updateQueue = current.updateQueue;
            
        // clone the dependencies object. this is mutated during the render phase,
        // so it cannot be shared with the current fiber.
        const currentDependencies = current.dependencies;
        workInProgress.dependencies = 
            currentDependencies === null
                ? null
                : {
                    expirationTime:currentDependencies.expirationTime,
                    firstContext:currentDependencies.firstContext,
                    responders:currentDependencies.responders
                };
        
        // these will be overridden during the paren's reconciliation
        workInProgress.sibling = current.sibling;
        workInProgress.index = current.index;
        workInProgress.ref = current.ref;
            
        if(enableProfilerTimer){
            workInProgress.selfBaseDuration = current.selfBaseDuration;
            workInProgress.treeBaseDuration = current.treeBaseDuration;
        }
        
        reurn workInProgress;
    }
}

`createWorkInProgress`

In [ ]:
{
    //used to reuse a fiber for a second pass.
    function resetWorkInProgress(
        workInProgress:Fiber,
        renderExpirationime:ExpirationTime
    ){
        //this resets the fiber to what createFiber or createWorkInProgress would
        //have set the values to before during the first pass. ideally this wouldn't
        //be necessary but unfortunately many code paths reads from the workInProgress
        // when they should be reading from current and writeing to workInProgress.
        
        //we assume pendingProps, index, key, ref, return are still untoched to
        //avoid doing another reconciliation.
            
        //reset the effect tag but keep any placement tags, since that's something
        // that child fiber is setting, not the reconciliation.
        workInProgress.effectTag &= Placement;
            
        // the effect list is no longer valid.
        workInProgress.nextEffect = null;
        workInProgress.firstEffect = null;
        workInProgress.lasEffect = null;
            
        let current = workInProgress.alternate;
        if(current === null) {
            // Reset to createFiber's initial values.
            workInProgress.childExpirationTime = NoWork;
            workInProgress.expirationTime = renderExpirationTime;
            
            workInProgress.child = null;
            workInProgress.memoizedProps = null;
            workInProgress.memoizedSate = null;
            workInProgress.updateQueue = null;
            
            workInProgress.dependencies = null;
            
            if(enableProfilerTimer){
                //Note: we don't reset the actualTime counts. it's useful to accumulate
                //actual time across multiple render passes.
                workInProgress.selfBaseDuration = 0;
                workInProgress.treeBaseDuration = 0;
            }
        }else{
            //reset to the cloned values that createWorkInProgress would've.
            workInProgress.childExpirationTime = current.childExpirationTime;
            workInProgress.expirationTime = current.expirationTime;
            
            workInProgress.child = current.child;
            workInProgress.memoizedProps = current.memoizedProps;
            workInProgress.memoizedSate = current.memoizedSate;
            workInProgress.updateQueue = current.updateQueue;
            
            //clone the dependencies object. this is mutated during the render phase, so
            // it annot be shared with the current fiber.
            const currentDependencies = current.dependencies;
            workInProgress.dependencies =
                currentDependencies === null
                    ? null
                    : {
                        expirationTime:currentDependencies.expirationTime,
                        firstContext:currentDependencies.firstContext,
                        responders:currentDependencies.responders
                    };
            
            if(enableProfilerTimer){
                //mote :we don't reset the actualTime counts. it's useful to acumulate
                // actual time across multiple render passes.
                workInProgress.selfBaseDuration = current.selfBaseDuration;
                workInProgress.treeBaseDuration = current.treeBAseDuration;
            }
        }
        
        return workInProgress;        
    }
}

`resetWorkInProgress`

In [ ]:
{
    function createHostRootFiber(tag:RootTag):Fiber{
        let mode;
        if(tag === ConcurrentRoot){
            mode = ConcurrentMode | BatchedMode | StricuMode;
        }else if(tag === BatchedRoot){
            mode = BatchedMode | StrictMode;
        }else{
            mode = NoMode;
        }
        
        if(enableProfilerTimer && isDevToolsPresent){
            //Always collect profile timings when DevTools are present.
            //thsi enables DevTools to start capturing timing at any point-Without
            //some nodes in the tree having empty base times.
            mode |= ProfileMode;
        }
        
        return createFiber(HostRoot,null,null,mode);
    }
    
    function createFiberFromTypeAndProps(
        type:any, //React$ElementType
        key:null | string,
        pendingProps:any,
        owner:null | Fiber,
        mode:TypeOfMode,
        expirationTime:ExpirationTime
    ):Fiber {
            let fiber;
            
            let fiberTag = IndeterminateComponent;
            //the resolved type is set if we know what the final type will be.it's not lazy.
            let resolvedType = type;
            if(typeof type === 'function'){
                if(shouldConstruct(type)){
                    fiberTag = ClassComponent;
                }
            }else if(tpyeof type === 'string'){
                fiberTag = HostComponent;
            }else{
                getTag:switch(type){
                    case REACT_FRAGMENT_TYPE:
                        return createFiberFromFragment(
                            pendingProps.children,
                            mode,
                            expirationTime,
                            key
                        )
                    case REACT_CONCURRENT_MODE_TYPE:
                        fiberTag = Mode;
                        mode |= ConcurrentMode | BatchedMode | StrictMode;
                        break;
                    case REACT_STRICT_MODE_TYPE:
                        fiberTag = Mode;
                        mode |= StrictMode;
                        break;
                    case REACT_PROFILER_TYPE:
                        return createFiberFromProfiler(pendingProps,mode,expirationTime,key);
                    case REACT_SUSPENSE_TYPE:
                        return createFiberFromSuspense(pendingProps,mode,expirationTime,key);
                    case REACT_SUSPENSE_LIST_TYPE:
                        return createFiberFromSuspenseList(
                            pendngProps,
                            mode,
                            expirationTime,
                            key
                        );
                    default:{
                        if(typeof type === 'object' && type!==null){
                            switch(type.$$typeof){
                                case REACT_PROVIDER_TYPE:
                                    fiberTag = ContextProvider;
                                    break getTag;
                                case REACT_CONTEXT_TYPE:
                                    //this is a consumer
                                    fiberTag = ContextConsumer;
                                    break getTag;
                                case REACT_FORWARD_REF_TYPE:
                                    fiberTag = ForwardRef;
                                    break getTag;
                                case REACT_MEMO_TYPE:
                                    fiberTag = MemoComponent;
                                    break getTag;
                                case REACT_LAZY_TYPE:
                                    fiberTag = LazyComponent;
                                    resolvedType = null;
                                    break getTag;
                                case REACT_FUNDAMENTAL_TYPE:
                                    if(enableFundamentalAPI){
                                        return createFiberFromFundamental(
                                            type,
                                            pendingProps,
                                            mode,
                                            expirationTime,
                                            key
                                        )
                                    }
                                    break;
                                case REACT_SCOPE_TYPE:
                                    if(enableScopeAPI){
                                        return createFiberFromScope(
                                            type,
                                            pendingProps,
                                            mode,
                                            expirationTime,
                                            key
                                        );
                                    }
                            }
                        }
                        let info = '';
                        invariant(
                          false,
                          'Element type is invalid: expected a string (for built-in ' +
                            'components) or a class/function (for composite components) ' +
                            'but got: %s.%s',
                          type == null ? type : typeof type,
                          info,
                        );
                    }
                }
            }
            
            fiber = createFiber(fiberTag,pendingProps,key,mode);
            fiber.elementType = type;
            fiber.type = resolvedType;
            fiber.expirationTie = expirationTime;
            
            return fiber;
        }
    
    function createFiberFromElement(
        element:ReatElement,
        mode:TypeOfMode,
        expirationTime:ExpirationTime
    ):Fiber {
            let owner = null;
            const type = element.type;
            const key = element.key;
            const pendingProps = element.props;
            const fiber = createFiberFromTypeAdnProps(
                type,
                key,
                pendingProps,
                owner,
                mode,
                expirationTime
            );
            return fiber;
        }
    
    function createFiberFromFragment(
        elements:ReactFragment,
        mode: TypeOfMode,
        expirationTime:ExpirationTime,
        key: null | string
    ):Fiber{
            const fiber = createFiber(Fragment,elements,key,mode);
            fiber.expirationTime = expirationTime;
            return fiber;
        }
    
    function createFiberFromFundamental(
        fundamentalComponent:ReactFundamentalComponent<any,any>,
        pendingProps:any,
        mode:TypeOfMode,
        expirationTime:ExpirationTime,
        key:null | string
    ):Fiber{
        const fiber = createFiber(FundamentalComponent,pendingProps,key,mode);
        fiber.elementTpe = fundamentalComponent;
        fiber.type = fundamentalComponent;
        fiber.expirationTime = expirationTime;
        return fiber;
    }
    
    function createFiberFromScope(
        scope:ReactScope,
        pendingProps:any,
        mode:TypeOfMode,
        expirationTime:ExpirationTime,
        key:null | string
    ):Fiber{
            const fiber = createFiber(ScopeComponent,pendingProps,key,mode);
            fiber.type = scope;
            fiber.elementType = scope;
            fiber.expirationTime = expirationTime;
            return fiber;
        }
    
    function createFiberFromProfiler(
        pendingProps:any,
        mode:TypeOfMode,
        expirationTime:ExpirationTime,
        key:null | string
    ):Fiber{
            const fiber = createFiber(Profiler,pendingProps,key,mode|ProfileMode);
            //todo the profiler fiber shouldn't hava a type,it has a tag.
            fiber.elementType = REACT_PROFILER_TYPE;
            fiber.type = REACT_PROFILER_TYPE;
            fiber.expirationTime = expirationTime;
            
            return fiber;
        }
    
    function createFiberFromSuspense(
        pendingProps:any,
        mode:TypeOfMode,
        expirationTime:ExpirationTime,
        key:null |string
    ):Fiber{
            const fiber = createFiber(SuspenseComponent, pendingProps,key,mode);
            
            //todo :the suspenseComponent fiber shouldn't hava a type . it has a tag.
            // this needs to be fixed in getComponentName se that is relies on the tag
            // instead.
            fiber.type = REACT_SUSPENSE_TYPE;
            fiber.elementType = REACT_SUSPENSE_TYPE;
            
            fiber.expirationTime = expirationTime;
            return fiber;
        }
    
    function createFiberFroSuspenseList(
        pendingProps:any,
        mode:TypeOfMode,
        expirationTime:ExpirationTime,
        key:null | string
    ):Fiber{
            const fiber = createFiber(SuspenseListComponent,pendingProps,key,mode);
            fiber.elementType = REACT_SUSPENSE_LIST_TYPE;
            fiber.expirationTime = expirationTime;
            return fiber;
        }
    
    function createFiberFromText(
        content:string,
        mode:TypeOfMode,
        expirationTime:ExprationTime
    ):Fiber{
            const fiber = reateFiber(HostText,content,null,mode);
            fiber.expirationTime = expirationTime;
            return fiber;
        }
    
    function createFiberFromHostInstanceForDeletion():Fiber{
        const fiber = createFiber(HostComponent,null,null,NoMode);
        //todo :these should not need a type
        fiber.elementType ='DELETED';
        fiber.type ='DELETED';
        return fiber;
    }
    
    function createFiberFromDehydratedFragment(
        dehydratedNode:SuspenseInstance
    ):Fiber{
            const fiber = createFiber(DehydratedFragment,null,null,NoMode);
            fiber.stateNode = dehydratedNode;
            return fiber;
        }
    
    function createFiberFromPortal(
        portal:ReactPortal,
        mode:TypeOfMode,
        expirationTime:ExpirationTime
    ):Fiber{
            const pendingProps = portal.children!==null?portal.children:[];
            const fiber = createFiber(HostPortal,pendingProps,portal.key,mode);
            fiber.expirationTime = expirationTime;
            fiber.stateNode = {
                containerInfo:portal.containerInfo,
                pendingChildren:null,//used by persistent updates
                implementation:portal.implementation
            };
            return fiber;
        }
    
    //used for stashing WIP properties to replay failed work in DEV.
    function assignFiberPropertiesInDEV(
        target:Fiber|null,
        source:Fiber
    ):Fiber{
            if(target === null){
                //this fiber's initial properties will always be overwritten.
                //we only use a fiber to ensure the same hidden class so DEV isn't slow;
                target= createFiber(IndeterminateComponent,null,null,NoMode);
            }
            
            //this is inientionally writen as a list of all properties.
            //we tried to use object.assign() instead but this is called in
            //this hottest path, and object.assign() was too slow:
            //https://github.com/facebook/react/issues/12502
            //this code is DEV-only so size is not concern.
              target.tag = source.tag;
              target.key = source.key;
              target.elementType = source.elementType;
              target.type = source.type;
              target.stateNode = source.stateNode;
              target.return = source.return;
              target.child = source.child;
              target.sibling = source.sibling;
              target.index = source.index;
              target.ref = source.ref;
              target.pendingProps = source.pendingProps;
              target.memoizedProps = source.memoizedProps;
              target.updateQueue = source.updateQueue;
              target.memoizedState = source.memoizedState;
              target.dependencies = source.dependencies;
              target.mode = source.mode;
              target.effectTag = source.effectTag;
              target.nextEffect = source.nextEffect;
              target.firstEffect = source.firstEffect;
              target.lastEffect = source.lastEffect;
              target.expirationTime = source.expirationTime;
              target.childExpirationTime = source.childExpirationTime;
              target.alternate = source.alternate;
              if (enableProfilerTimer) {
                target.actualDuration = source.actualDuration;
                target.actualStartTime = source.actualStartTime;
                target.selfBaseDuration = source.selfBaseDuration;
                target.treeBaseDuration = source.treeBaseDuration;
              }
              target._debugID = source._debugID;
              target._debugSource = source._debugSource;
              target._debugOwner = source._debugOwner;
              target._debugIsCurrentlyTiming = source._debugIsCurrentlyTiming;
              target._debugNeedsRemount = source._debugNeedsRemount;
              target._debugHookTypes = source._debugHookTypes;
              return target;
        }
}